# Herramienta de visualización
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

Un extra que se presenta aquí es la posibilidad de aplicar la solución encontrada por nuestro algoritmo. Para ello, se creó un visualizador basado en [PyGame](https://www.pygame.org/news), que representa gráficamente las Torres de Hanoi y ejecuta la secuencia indicada por la solución obtenida.

![files](./img/hanoi_sim.png)

La implementación se encuentra en el módulo `simulator` y está ampliamente documentada, por si les interesa profundizar en cómo fue desarrollada. Es altamente configurable, tanto en la velocidad de ejecución como en tamaños y geometrías de los discos, todos accesibles desde el archivo `constants.py`.

Para ejecutar la animación desde la línea de comandos, se debe utilizar el siguiente comando:

```bash
python3 ./simulation_hanoi.py

```

Para funcionar correctamente, el visualizador depende de dos archivos JSON que deben estar ubicados dentro del directorio `simulator`:

- `initial_state.json`: Define cómo se inicializan los discos y cuántos discos habrá en total.
- `sequence.json`: Define el orden de los movimientos de los discos durante la resolución.

El visualizador permite movimientos ilegales desde el punto de vista del juego, en el sentido de que no valida si se está siguiendo estrictamente la lógica de las Torres de Hanoi. Sin embargo, mantiene ciertas restricciones físicas, como la altura correcta al insertar un disco en una varilla. No puede realizar movimientos imposibles (por ejemplo, mover un disco que no está en la parte superior de su varilla o intentar retirar un disco inexistente). Aunque el visualizador no arrojará errores ante estas inconsistencias, la animación resultante podría no tener sentido.

## Requisitos para los programas de búsqueda de soluciones

Los programas utilizados para encontrar soluciones deben generar archivos con un formato específico para que puedan ser visualizados correctamente.

### `initial_state.json`

El programa de búsqueda debe generar (o se puede crear manualmente) un archivo JSON que represente el estado inicial de los discos. Se acepta cualquier configuración válida y hasta un máximo de 15 discos.

Formato esperado:

```JSON
{
  "peg_1": [5, 4, 3, 2, 1],
  "peg_2": [],
  "peg_3": []
}
```

Donde:

- `peg_1` es la varilla izquierda,
- `peg_2` la del medio,
- `peg_3` la derecha.

Cada varilla contiene una lista con los discos que tiene, ordenados de abajo hacia arriba. El disco más pequeño es el 1, y los números aumentan a medida que los discos son más grandes.

**Reglas:**
- No debe haber discos repetidos.
- Todos los discos intermedios deben estar presentes (por ejemplo, si está el disco 4, también deben estar los discos 1, 2 y 3).

**Ejemplo válido:**

```JSON
{
  "peg_1": [6, 2],
  "peg_2": [8, 7, 4],
  "peg_3": [1, 5, 3]
}
```

**Ejemplos inválidos:**

❌ Falta el disco 4:

```JSON
{
  "peg_1": [6, 2],
  "peg_2": [8, 7],
  "peg_3": [1, 5, 3]
}
```

❌ Disco 6 repetido:

```JSON
{
  "peg_1": [6, 2],
  "peg_2": [8, 7, 6],
  "peg_3": [1, 5, 3]
}
```

### `sequence.json`

El programa también debe generar un archivo con la secuencia de movimientos, donde cada acción consiste en mover un único disco desde una varilla a otra.

Formato esperado:

```JSON
[
	{
		"type": "movement",
		"disk": 1,
		"peg_start": 1,
		"peg_end": 2
	},
	{
		"type": "movement",
		"disk": 2,
		"peg_start": 1,
		"peg_end": 3
	},
        .
        .
        . 
	{
		"type": "movement",
		"disk": 3,
		"peg_start": 2,
		"peg_end": 1
	}
]
```

Cada elemento del arreglo representa un movimiento:
- `type`: debe ser `"movement"` (otros tipos serán ignorados).
- `disk`: número del disco que se mueve.
- `peg_start`: varilla desde donde se toma el disco.
- `peg_end`: varilla de destino.

---

## Usando la implementación que vimos en notebooks anteriores

La clase `NodeHanoi`, que utilizamos para construir los nodos del árbol de búsqueda, incluye un método llamado `generate_solution_for_simulator()`. Este método permite generar automáticamente ambos archivos JSON necesarios (`initial_state.json` y `sequence.json`), a partir del nodo actual.

Este método puede invocarse directamente desde cualquier nodo del árbol (por ejemplo, el nodo solución) y recorrerá hacia atrás hasta la raíz para reconstruir todo el camino de solución.

### Ejemplo de uso:

Vamos a reutilizar el algoritmo de búsqueda que implementamos en el notebook anterior:

In [1]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [ ]:
def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola FIFO con el nodo inicial
    frontier = [NodeHanoi(problem.initial)]  

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al conjunto para evitar duplicados
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si alcanzamos el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            if next_node.state not in explored:
                frontier.insert(0, next_node)

    # Si no se encontró la solución, devolvemos las métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth,  # OBS: si no se encuentra solución, este valor solo tiene sentido en BFS. En otros casos, se debe llevar un registro explícito de la profundidad máxima alcanzada.
        "cost_total": None,
    }
    return None, metrics

Ejecutemos una búsqueda

In [3]:
solution, metrics = breadth_first_search(number_disks=5)

Veamos algunas métricas obtenidas:

In [4]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 1351
states_visited: 233
nodes_in_frontier: 285
max_depth: 31
cost_total: 31.0


#### Generación de los archivos para el visualizador

Ahora ejecutamos el método que nos genera los archivos JSON necesarios para el simulador:

In [5]:
solution.generate_solution_for_simulator()

Este método no retorna ningún valor, pero guarda automáticamente los archivos `initial_state.json` y `sequence.json` en el directorio donde estamos ejecutando la notebook.

![files](./img/files.png)

Esos archivos `.json` deben moverse a la carpeta `simulator`, y desde esa misma carpeta podemos ejecutar el siguiente comando para visualizar la solución:

```bash
python3 ./simulation_hanoi.py

```
Esto lanzará la animación que representa gráficamente la solución encontrada para el problema de las Torres de Hanoi.